# Competitor Analysis Agentic Notebook (Google GenAI + SerpAPI)

This notebook searches the web for company facts and uses Google GenAI to extract and compare key business information.

In [ ]:
# 1. Install Required Libraries
!pip install google-generativeai google-search-results pandas

In [ ]:
# 2. Set Up API Keys
import os

os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"  # Google GenAI
os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY"  # SerpAPI

In [ ]:
# 3. Import Libraries
from google import genai
from google.genai import types
from serpapi import GoogleSearch
import pandas as pd
import time

In [ ]:
# 4. Authenticate Google GenAI
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-pro")

In [ ]:
# 5. Define Companies and Info Types
target_company = "OpenAI"
competitors = ["Anthropic", "Cohere", "Google DeepMind"]
info_types = [
    "headquarters location",
    "number of employees",
    "financials (revenue, funding, valuation)",
    "marketing strategy"
]

In [ ]:
# 6. Define Helper Functions
def serpapi_search(query):
    params = {
        "engine": "google",
        "q": query,
        "api_key": os.environ["SERPAPI_API_KEY"],
        "num": 5
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    snippets = []
    if "organic_results" in results:
        for r in results["organic_results"]:
            if "snippet" in r:
                snippets.append(r["snippet"])
    return "\n".join(snippets)

def extract_info_with_genai(company, info_type, context):
    prompt = (
        f"You are a business analyst. Extract the {info_type} for {company} "
        f"from the following web search results. If not found, say 'Not found.'\n\n"
        f"Web search results:\n{context}"
    )
    response = model.generate_content(prompt)
    return response.text.strip()

In [ ]:
# 7. Gather Data
report = {}
for company in [target_company] + competitors:
    report[company] = {}
    for info in info_types:
        print(f"Searching for {info} of {company}...")
        context = serpapi_search(f"{company} {info}")
        time.sleep(2)  # To avoid hitting rate limits
        info_extracted = extract_info_with_genai(company, info, context)
        report[company][info] = info_extracted

In [ ]:
# 8. Comparative Report Generation
import json

comparison_prompt = (
    "Given the following company data, write a detailed competitor analysis comparing the target company "
    "to its competitors. Highlight strengths, weaknesses, and differences in location, headcount, financials, "
    "and marketing strategy. Present the results in a structured, readable format.\n\n"
    f"{json.dumps(report, indent=2)}"
)

analysis = model.generate_content(comparison_prompt)
print(analysis.text)

In [ ]:
# 9. (Optional) Display as Table
df = pd.DataFrame(report).T
display(df)